# Entrainement model Lille avec ml flow 

In [1]:
# Installation des dépendances si nécessaire
# !pip install mlflow scikit-learn pandas numpy xgboost

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
import joblib

print("Imports réussis")

Imports réussis


## Configuration MlFlow 

In [3]:
# Configuration du tracking URI
mlflow.set_tracking_uri("file:../mlruns")

print("MLflow configure")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

MLflow configure
Tracking URI: file:../mlruns


## Chargement des données 

In [4]:
print("\n" + "="*60)
print("CHARGEMENT DONNEES BORDEAUX")
print("="*60)

# Charger toutes les données France
print("\nChargement des données France 2024...")
df_full = pd.read_csv("../data/Valeurs_foncieres-2024.txt", sep="|", low_memory=False)

# Filtrer département Gironde (33)
df_gironde = df_full[df_full['Code departement'] == '33'].copy()
print(f"Transactions département Gironde : {len(df_gironde)}")

# Trouver Bordeaux
print(f"\nRecherche de Bordeaux...")
df_bordeaux_search = df_gironde[df_gironde['Commune'].str.contains('BORDEAUX', case=False, na=False)]

if len(df_bordeaux_search) > 0:
    print(f"\nVilles trouvées contenant 'BORDEAUX' :")
    print(df_bordeaux_search.groupby(['Code commune', 'Commune']).size())
    
    # Prendre le code commune de Bordeaux
    code_bordeaux = df_bordeaux_search['Code commune'].mode()[0]
    print(f"\nCode commune Bordeaux : {code_bordeaux}")
    
    # Filtrer avec le bon code
    df_bordeaux = df_gironde[df_gironde['Code commune'] == code_bordeaux].copy()
    print(f"Transactions Bordeaux 2024 : {len(df_bordeaux)}")
    
    # Sauvegarder
    df_bordeaux.to_csv("../data/bordeaux_2024.csv", index=False)
    print("Fichier bordeaux_2024.csv cree")
else:
    print("Bordeaux non trouve dans les données ")


CHARGEMENT DONNEES BORDEAUX

Chargement des données France 2024...
Transactions département Gironde : 82824

Recherche de Bordeaux...

Villes trouvées contenant 'BORDEAUX' :
Code commune  Commune               
13            ARTIGUES-PRES-BORDEAUX      261
63            BORDEAUX                  11660
99            CARIGNAN-DE-BORDEAUX         90
245           LIGNAN DE BORDEAUX           18
381           ST CAPRAIS DE BORDEAUX       97
dtype: int64

Code commune Bordeaux : 63
Transactions Bordeaux 2024 : 11660
Fichier bordeaux_2024.csv cree


## Filtrage et nettoyage 

In [ ]:
# Fonction de nettoyage 
def select_features_and_clean(df, dataset_name):
    """Sélectionner les colonnes et nettoyer"""
    print(f"\nNettoyage donnees {dataset_name}...")
    
    if len(df) == 0:
        print(f"Aucune donnee pour {dataset_name}")
        return None, None, None
    
    # Colonnes à conserver
    required_cols = ['Surface reelle bati', 'Nombre pieces principales', 'Type local', 'Valeur fonciere']
    optional_cols = ['Surface terrain', 'Nombre de lots']
    
    available_cols = [col for col in required_cols if col in df.columns]
    available_optional = [col for col in optional_cols if col in df.columns]
    
    selected_cols = available_cols + available_optional
    df_clean = df[selected_cols].copy()
    
    # CONVERTIR LES COLONNES NUMÉRIQUES
    numeric_cols = ['Surface reelle bati', 'Valeur fonciere', 'Surface terrain', 'Nombre de lots', 'Nombre pieces principales']
    for col in numeric_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.replace(',', '.').replace('', '0')
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Créer prix_m2
    df_clean['prix_m2'] = df_clean['Valeur fonciere'] / df_clean['Surface reelle bati']
    
    print(f"   Avant nettoyage : {len(df_clean)} lignes")
    
    # Remplacer valeurs manquantes
    if 'Surface terrain' in df_clean.columns:
        df_clean['Surface terrain'] = df_clean['Surface terrain'].fillna(0)
    if 'Nombre de lots' in df_clean.columns:
        df_clean['Nombre de lots'] = df_clean['Nombre de lots'].fillna(1)
    
    # Supprimer lignes avec données essentielles manquantes
    df_clean = df_clean.dropna(subset=['Surface reelle bati', 'Valeur fonciere', 'prix_m2'])
    
    # Retirer outliers (IQR)
    if len(df_clean) > 0:
        Q1 = df_clean['prix_m2'].quantile(0.25)
        Q3 = df_clean['prix_m2'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        mask = (df_clean['prix_m2'] >= lower_bound) & (df_clean['prix_m2'] <= upper_bound)
        df_clean = df_clean[mask]
    
    print(f"   Apres nettoyage : {len(df_clean)} lignes")
    
    # Préparer X et y
    feature_cols = [col for col in df_clean.columns if col not in ['Valeur fonciere', 'prix_m2', 'Type local']]
    X = df_clean[feature_cols]
    y = df_clean['prix_m2']
    
    print(f"   Prix m2 moyen: {df_clean['prix_m2'].mean():.2f} EUR")
    print(f"   Variables : {list(X.columns)}")
    
    return X, y, df_clean

# Charger Bordeaux
print("\n" + "="*60)
print("PREPARATION DONNEES BORDEAUX")
print("="*60)

df_bdx = pd.read_csv("../data/bordeaux_2024.csv")
print(f"Donnees chargees : {len(df_bdx)} transactions")

# Filtrer 4 pièces
df_bdx_4p = df_bdx[df_bdx['Nombre pieces principales'] == 4.0].copy()
print(f"Logements 4 pieces : {len(df_bdx_4p)}")

# Séparation appartements / maisons
appartements_bdx = df_bdx_4p[df_bdx_4p['Type local'] == 'Appartement'].copy()
maisons_bdx = df_bdx_4p[df_bdx_4p['Type local'] == 'Maison'].copy()

print(f"\nAppartements 4 pieces : {len(appartements_bdx)}")
print(f"Maisons 4 pieces : {len(maisons_bdx)}")

# Nettoyage
print("\n" + "="*50)
print("NETTOYAGE BORDEAUX - APPARTEMENTS")
print("="*50)
X_apt_bdx, y_apt_bdx, df_apt_bdx_clean = select_features_and_clean(appartements_bdx, "APPARTEMENTS BDX")

print("\n" + "="*50)
print("NETTOYAGE BORDEAUX - MAISONS")
print("="*50)
X_maisons_bdx, y_maisons_bdx, df_maisons_bdx_clean = select_features_and_clean(maisons_bdx, "MAISONS BDX")


PREPARATION DONNEES BORDEAUX
Donnees chargees : 11660 transactions
Logements 4 pieces : 747

Appartements 4 pieces : 480
Maisons 4 pieces : 267

NETTOYAGE BORDEAUX - APPARTEMENTS

Nettoyage donnees APPARTEMENTS BDX...
   Avant nettoyage : 480 lignes
   Apres nettoyage : 396 lignes
   Prix m2 moyen: 3986.15 EUR
   Variables : ['Surface reelle bati', 'Nombre pieces principales', 'Surface terrain', 'Nombre de lots']

NETTOYAGE BORDEAUX - MAISONS

Nettoyage donnees MAISONS BDX...
   Avant nettoyage : 267 lignes
   Apres nettoyage : 260 lignes
   Prix m2 moyen: 4891.06 EUR
   Variables : ['Surface reelle bati', 'Nombre pieces principales', 'Surface terrain', 'Nombre de lots']


## Chargement du moèle lille depuis MlFlow 

In [6]:
print("\n" + "="*60)
print("CHARGEMENT DES MODELES LILLE DEPUIS MLFLOW")
print("="*60)

# Configuration tracking URI
mlflow.set_tracking_uri("file:./mlruns")

# Run IDs des modèles Lille
apt_run_id = "cc34f04d13ee4fd3b984a7d16ad7d919"
maisons_run_id = "a414d25447634108971c44dbf5e80c93"

# Charger modèle et scaler appartements
print("\nChargement modele appartements Lille...")
model_apt_lille = mlflow.sklearn.load_model(f"runs:/{apt_run_id}/model")
scaler_apt_lille = mlflow.sklearn.load_model(f"runs:/{apt_run_id}/scaler")
print("Modele appartements charge")

# Charger modèle et scaler maisons
print("\nChargement modele maisons Lille...")
model_maisons_lille = mlflow.sklearn.load_model(f"runs:/{maisons_run_id}/model")
scaler_maisons_lille = mlflow.sklearn.load_model(f"runs:/{maisons_run_id}/scaler")
print("Modele maisons charge")

print("\nTous les modeles Lille sont charges !")


CHARGEMENT DES MODELES LILLE DEPUIS MLFLOW

Chargement modele appartements Lille...
Modele appartements charge

Chargement modele maisons Lille...
Modele maisons charge

Tous les modeles Lille sont charges !


## Test de generalisation sur bordeaux 

In [7]:
print("\n" + "="*60)
print("TEST GENERALISATION - APPARTEMENTS")
print("="*60)

# Standardiser les données Bordeaux avec le scaler de Lille
X_apt_bdx_scaled = scaler_apt_lille.transform(X_apt_bdx)

# Prédire avec le modèle Lille
y_pred_apt_bdx = model_apt_lille.predict(X_apt_bdx_scaled)

# Calculer les métriques
mse_apt_bdx = mean_squared_error(y_apt_bdx, y_pred_apt_bdx)
rmse_apt_bdx = np.sqrt(mse_apt_bdx)
r2_apt_bdx = r2_score(y_apt_bdx, y_pred_apt_bdx)

print("\nResultats Appartements sur Bordeaux :")
print(f"MSE:  {mse_apt_bdx:>12,.2f}")
print(f"RMSE: {rmse_apt_bdx:>12,.2f} EUR/m2")
print(f"R2:   {r2_apt_bdx:>12.3f}")

print("\nComparaison Lille vs Bordeaux (Appartements) :")
print(f"  Lille Test RMSE:    1,022.54 EUR/m2")
print(f"  Bordeaux Test RMSE: {rmse_apt_bdx:>8,.2f} EUR/m2")
print(f"  Ecart: {abs(rmse_apt_bdx - 1022.54):>8,.2f} EUR/m2")


TEST GENERALISATION - APPARTEMENTS

Resultats Appartements sur Bordeaux :
MSE:  2,856,632.60
RMSE:     1,690.16 EUR/m2
R2:         -0.286

Comparaison Lille vs Bordeaux (Appartements) :
  Lille Test RMSE:    1,022.54 EUR/m2
  Bordeaux Test RMSE: 1,690.16 EUR/m2
  Ecart:   667.62 EUR/m2


## Test de généralisation sur Bordeaux - Maisons


In [8]:
print("\n" + "="*60)
print("TEST GENERALISATION - MAISONS")
print("="*60)

# Standardiser les données Bordeaux avec le scaler de Lille
X_maisons_bdx_scaled = scaler_maisons_lille.transform(X_maisons_bdx)

# Prédire avec le modèle Lille
y_pred_maisons_bdx = model_maisons_lille.predict(X_maisons_bdx_scaled)

# Calculer les métriques
mse_maisons_bdx = mean_squared_error(y_maisons_bdx, y_pred_maisons_bdx)
rmse_maisons_bdx = np.sqrt(mse_maisons_bdx)
r2_maisons_bdx = r2_score(y_maisons_bdx, y_pred_maisons_bdx)

print("\nResultats Maisons sur Bordeaux :")
print(f"MSE:  {mse_maisons_bdx:>12,.2f}")
print(f"RMSE: {rmse_maisons_bdx:>12,.2f} EUR/m2")
print(f"R2:   {r2_maisons_bdx:>12.3f}")

print("\nComparaison Lille vs Bordeaux (Maisons) :")
print(f"  Lille Test RMSE:    876.63 EUR/m2")
print(f"  Bordeaux Test RMSE: {rmse_maisons_bdx:>8,.2f} EUR/m2")
print(f"  Ecart: {abs(rmse_maisons_bdx - 876.63):>8,.2f} EUR/m2")


TEST GENERALISATION - MAISONS

Resultats Maisons sur Bordeaux :
MSE:  6,287,102.53
RMSE:     2,507.41 EUR/m2
R2:         -1.902

Comparaison Lille vs Bordeaux (Maisons) :
  Lille Test RMSE:    876.63 EUR/m2
  Bordeaux Test RMSE: 2,507.41 EUR/m2
  Ecart: 1,630.78 EUR/m2


## Logger les résultats dans MLflow

In [9]:
print("\n" + "="*60)
print("LOGGING RESULTATS BORDEAUX DANS MLFLOW")
print("="*60)

# Logger les tests sur Bordeaux
with mlflow.start_run(run_name="test_generalisation_bordeaux") as run:
    
    # Paramètres
    mlflow.log_param("test_city", "bordeaux")
    mlflow.log_param("train_city", "lille")
    mlflow.log_param("n_samples_apt_bdx", len(X_apt_bdx))
    mlflow.log_param("n_samples_maisons_bdx", len(X_maisons_bdx))
    
    # Métriques appartements
    mlflow.log_metric("apt_bdx_mse", mse_apt_bdx)
    mlflow.log_metric("apt_bdx_rmse", rmse_apt_bdx)
    mlflow.log_metric("apt_bdx_r2", r2_apt_bdx)
    mlflow.log_metric("apt_rmse_degradation", rmse_apt_bdx - 1022.54)
    
    # Métriques maisons
    mlflow.log_metric("maisons_bdx_mse", mse_maisons_bdx)
    mlflow.log_metric("maisons_bdx_rmse", rmse_maisons_bdx)
    mlflow.log_metric("maisons_bdx_r2", r2_maisons_bdx)
    mlflow.log_metric("maisons_rmse_degradation", rmse_maisons_bdx - 876.63)
    
    print("Resultats logges dans MLflow")
    print(f"Run ID: {run.info.run_id}")

print("\nPhase 2 terminee !")


LOGGING RESULTATS BORDEAUX DANS MLFLOW
Resultats logges dans MLflow
Run ID: c4d789c8f3c744e3b3d210e0d59fff9b

Phase 2 terminee !
